In [1]:
import yfinance as yf
import ccxt
import aiohttp
import asyncio
import pandas as pd
from datetime import datetime, timedelta

# Data Retrieval Tools

In [2]:
class DataRetrievalAgent:
    def __init__(self):
        self.crypto_exchange = ccxt.binance()  # Using Binance for crypto data

    async def fetch_stock_data(self, ticker, period="1d", interval="1h"):
        """
        Fetches stock data from Yahoo Finance.
        """
        try:
            stock = yf.Ticker(ticker)
            data = stock.history(period=period, interval=interval)
            return {
                "ticker": ticker,
                "latest_price": data["Close"].iloc[-1] if not data.empty else None,
                "high": data["High"].iloc[-1] if not data.empty else None,
                "low": data["Low"].iloc[-1] if not data.empty else None,
                "volume": data["Volume"].iloc[-1] if not data.empty else None,
                "timestamp": str(datetime.now())
            }
        except Exception as e:
            return {"error": f"Stock data retrieval failed: {str(e)}"}

    async def fetch_crypto_data(self, symbol="BTC/USDT"):
        """
        Fetches real-time crypto data from Binance.
        """
        try:
            ticker = self.crypto_exchange.fetch_ticker(symbol)
            return {
                "symbol": symbol,
                "latest_price": ticker["last"],
                "high": ticker["high"],
                "low": ticker["low"],
                "volume": ticker["quoteVolume"],
                "timestamp": ticker["timestamp"]
            }
        except Exception as e:
            return {"error": f"Crypto data retrieval failed: {str(e)}"}

    async def fetch_historical_stock(self, ticker, days=30):
        """
        Fetches historical stock data for trend analysis.
        """
        try:
            end_date = datetime.today().strftime("%Y-%m-%d")
            start_date = (datetime.today() - timedelta(days=days)).strftime("%Y-%m-%d")
            stock = yf.Ticker(ticker)
            data = stock.history(start=start_date, end=end_date)
            return data.to_dict(orient="records") if not data.empty else {"error": "No historical data available"}
        except Exception as e:
            return {"error": f"Historical stock data retrieval failed: {str(e)}"}

    async def fetch_historical_crypto(self, symbol="BTC/USDT", days=30):
        """
        Fetches historical crypto data using Binance.
        """
        try:
            since = self.crypto_exchange.parse8601((datetime.utcnow() - timedelta(days=days)).isoformat())
            candles = self.crypto_exchange.fetch_ohlcv(symbol, timeframe="1d", since=since, limit=days)
            return [{"timestamp": c[0], "open": c[1], "high": c[2], "low": c[3], "close": c[4], "volume": c[5]} for c in candles]
        except Exception as e:
            return {"error": f"Historical crypto data retrieval failed: {str(e)}"}

    async def run(self, stock_ticker="TSLA", crypto_symbol="BTC/USDT"):
        """
        Runs the agent to fetch real-time and historical data.
        """
        tasks = [
            self.fetch_stock_data(stock_ticker),
            self.fetch_crypto_data(crypto_symbol),
            self.fetch_historical_stock(stock_ticker, days=30),
            self.fetch_historical_crypto(crypto_symbol, days=30),
        ]
        stock, crypto, stock_hist, crypto_hist = await asyncio.gather(*tasks)
        return {
            "stock_data": stock,
            "crypto_data": crypto,
            "historical_stock": stock_hist,
            "historical_crypto": crypto_hist
        }

In [3]:
# Testing in Jupyter Notebook
async def main():
    agent = DataRetrievalAgent()
    result = await agent.run(stock_ticker="AAPL", crypto_symbol="ETH/USDT")
    print(result)

# Run in async mode
await main()

/var/folders/dd/1frb69z52qjck_t8hrk9lt2w0000gn/T/ipykernel_42323/59532301.py:58: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  since = self.crypto_exchange.parse8601((datetime.utcnow() - timedelta(days=days)).isoformat())


{'stock_data': {'ticker': 'AAPL', 'latest_price': np.float64(227.63999938964844), 'high': np.float64(227.8800048828125), 'low': np.float64(227.1999969482422), 'volume': np.int64(3507517), 'timestamp': '2025-02-10 21:59:01.756084'}, 'crypto_data': {'symbol': 'ETH/USDT', 'latest_price': 2664.82, 'high': 2693.79, 'low': 2527.2, 'volume': 1217004049.62438, 'timestamp': 1739224744052}, 'historical_stock': [{'Open': 233.52999877929688, 'High': 234.6699981689453, 'Low': 229.72000122070312, 'Close': 234.39999389648438, 'Volume': 49630700, 'Dividends': 0.0, 'Stock Splits': 0.0}, {'Open': 234.75, 'High': 236.1199951171875, 'Low': 232.47000122070312, 'Close': 233.27999877929688, 'Volume': 39435300, 'Dividends': 0.0, 'Stock Splits': 0.0}, {'Open': 234.63999938964844, 'High': 238.9600067138672, 'Low': 234.42999267578125, 'Close': 237.8699951171875, 'Volume': 39832000, 'Dividends': 0.0, 'Stock Splits': 0.0}, {'Open': 237.35000610351562, 'High': 238.00999450683594, 'Low': 228.02999877929688, 'Close':

## Test

In [4]:
import asyncio

async def test_versatility():
    agent = DataRetrievalAgent()

    # Multiple stock symbols
    stock_symbols = ["AAPL", "TSLA", "NVDA", "AMZN"]
    crypto_symbols = ["BTC/USDT", "ETH/USDT", "SOL/USDT"]

    # Run tests
    tasks = []
    for stock in stock_symbols:
        tasks.append(agent.fetch_stock_data(stock, period="5d", interval="1h"))

    for crypto in crypto_symbols:
        tasks.append(agent.fetch_crypto_data(crypto))

    results = await asyncio.gather(*tasks)

    # Print results
    for i, stock in enumerate(stock_symbols):
        print(f"Stock Data [{stock}]:", results[i])

    for j, crypto in enumerate(crypto_symbols):
        print(f"Crypto Data [{crypto}]:", results[len(stock_symbols) + j])

# Run tests
await test_versatility()

Stock Data [AAPL]: {'ticker': 'AAPL', 'latest_price': np.float64(227.63999938964844), 'high': np.float64(227.8800048828125), 'low': np.float64(227.1999969482422), 'volume': np.int64(3507517), 'timestamp': '2025-02-10 22:02:31.607483'}
Stock Data [TSLA]: {'ticker': 'TSLA', 'latest_price': np.float64(350.6400146484375), 'high': np.float64(355.0), 'low': np.float64(350.510009765625), 'volume': np.int64(8336029), 'timestamp': '2025-02-10 22:02:31.675644'}
Stock Data [NVDA]: {'ticker': 'NVDA', 'latest_price': np.float64(133.50999450683594), 'high': np.float64(134.6699981689453), 'low': np.float64(133.38999938964844), 'volume': np.int64(17439170), 'timestamp': '2025-02-10 22:02:31.744475'}
Stock Data [AMZN]: {'ticker': 'AMZN', 'latest_price': np.float64(233.1300048828125), 'high': np.float64(233.9149932861328), 'low': np.float64(232.97999572753906), 'volume': np.int64(2887593), 'timestamp': '2025-02-10 22:02:31.857721'}
Crypto Data [BTC/USDT]: {'symbol': 'BTC/USDT', 'latest_price': 97339.63,

In [5]:
async def test_invalid_inputs():
    agent = DataRetrievalAgent()
    
    # Invalid stock and crypto symbols
    stock_invalid = await agent.fetch_stock_data("INVALID")
    crypto_invalid = await agent.fetch_crypto_data("FAKE/USDT")

    print("Invalid Stock Test:", stock_invalid)
    print("Invalid Crypto Test:", crypto_invalid)

await test_invalid_inputs()

$INVALID: possibly delisted; no price data found  (period=1d) (Yahoo error = "No data found, symbol may be delisted")


Invalid Stock Test: {'ticker': 'INVALID', 'latest_price': None, 'high': None, 'low': None, 'volume': None, 'timestamp': '2025-02-10 22:03:04.094932'}
Invalid Crypto Test: {'error': 'Crypto data retrieval failed: binance does not have market symbol FAKE/USDT'}
